In [47]:
pip install google-search-results
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [2]:
from serpapi import GoogleSearch
import googlemaps
import csv
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import pandas as pd

from tqdm.notebook import tqdm
tqdm.pandas()

serp_api_key='8a8e3b429e2fc84b99fe82f5093a4644947098bd07f9ec44259dca8f8de4e85c'



In [4]:
def search_results(google_search):
    params = {
      "engine": "google",
      "q": google_search+"email",
      "api_key": serp_api_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    return results

In [48]:
def get_place_id(results):  
    try: 
        return results['knowledge_graph']['place_id']
    except:
        return "No place found"

def get_place_address(results):
    try:
        return results['knowledge_graph']['address']
    except:
        return "No place found"

def get_shop_name(results):
    try: 
        return results['knowledge_graph']['title']
    except:
        return "No place found"

def get_search_link(results):
    try: 
        return results['knowledge_graph']['knowledge_graph_search_link']
    except:
        return "No place found"

def price_lookup(results):
    try:
        return results['knowledge_graph']['price']
    except:
        return "No price found"

def googleclassification_lookup(results):
    try:
        return results['knowledge_graph']['type']
    except:
        return "No Google Classification found"
    
def reservation_type(results):
    try:
        return results['knowledge_graph']['reservation_providers'][0]['name']
    except:
        return "No reservation provider found"

In [9]:
def email_lookup(results):
    try:
        email1=results['organic_results'][0]['snippet_highlighted_words']
    except:
        email1="No email found"
    try:
        email2=results['organic_results'][1]['snippet_highlighted_words']
    except:
        email2="No email found"
    try:
        return email1,email2 
    except:
        return "No email found"


In [11]:
def review_audit(place_id,max_reviews):

    reviews = fetch_reviews(place_id, serp_api_key, max_reviews)

# Print out the reviews
    review_count=0
    reviews_with_pizza=0
    for review in reviews:
        if len(review['snippet'])>1:
            review_count+=1
            if 'pizza' in review['snippet'].lower() or 'pie' in review['snippet'].lower() or 'pizzeria' in review['snippet'].lower() or 'slice' in review['snippet'].lower():
                reviews_with_pizza+=1
    return [review_count ,reviews_with_pizza]

In [12]:
def fetch_reviews(place_id, api_key, max_reviews=18):
    all_reviews = []
    next_page_token = None

    while len(all_reviews) < max_reviews:
        params = {
            "engine": "google_maps_reviews",
            "place_id": place_id,
            "api_key": api_key,
            "hl": "en",  # Language (optional),
            "sort_by": "qualityScore"
        }

        if next_page_token:
            params["next_page_token"] = next_page_token

        search = GoogleSearch(params)
        results = search.get_dict()

        if "reviews" in results:
            all_reviews.extend(results["reviews"])

        if "serpapi_pagination" in results and "next_page_token" in results["serpapi_pagination"]:
            next_page_token = results["serpapi_pagination"]["next_page_token"]
        else:
            break

    return all_reviews[:18]





In [13]:
def process_csv(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    df['results']=df['Google Search'].progress_apply(search_results)
    df['PlaceID'] = df['results'].apply(get_place_id)
    df['Shop Name']=df['results'].apply(get_shop_name)
    df['Address']=df['results'].apply(get_place_address)
    df['output'] = df.progress_apply(lambda row:review_audit(row['PlaceID'],18),axis=1)
    df['Review_counts']= df['output'].apply(lambda x: pd.Series(x))[0]
    df['Reviews_with_pizza']= df['output'].apply(lambda x: pd.Series(x))[1]
    df['TP_review_audit']=(df['Reviews_with_pizza']/df['Review_counts']).apply(lambda x: f"{int(np.nan_to_num(x * 100))}%")
    df['Reservation Provider']=df['results'].apply(reservation_type)
    df['Email']=df['results'].apply(email_lookup)
    df['Email1']= df['Email'].apply(lambda x: pd.Series(x))[0]
    df['Email2']= df['Email'].apply(lambda x: pd.Series(x))[1]
    df['Price']=df['results'].apply(price_lookup)
    df['Google Classification']=df['results'].apply(googleclassification_lookup)
    df.drop(['results','Email','output'], axis=1,inplace=True)
    df.to_csv(file_path[:-4]+" - output.csv", index=False)
    return df

In [ ]:
%%time
test=process_csv('Batch 1.csv')


**Use the following for processing large API requests**

In [14]:
# Asynchronous function to fetch API results from Google Places API
SERP_API_URL="https://serpapi.com/search"

async def async_search_results(session, google_search):
    params = {
        'q': google_search+" email",
        'api_key': serp_api_key,
        'engine': 'google'
    }
    async with session.get(SERP_API_URL, params=params) as response:
        data = await response.json()
        return data

# Asynchronous function to handle multiple requests
async def process_async_search_results(df):
    async with aiohttp.ClientSession() as session:
        tasks = []
        # Use tqdm to iterate over queries with progress bar
        for query in tqdm(df['Google Search'], total=len(df['Google Search'])):
            tasks.append(async_search_results(session, query))
        return await asyncio.gather(*tasks, return_exceptions=True)

In [15]:
# Asynchronous function to fetch reviews from Google Reviews API
async def async_fetch_reviews(place_id, api_key, max_reviews=18):
    all_reviews = []
    next_page_token = None

    async with aiohttp.ClientSession() as session:
        while len(all_reviews) < max_reviews:
            params = {
                "engine": "google_maps_reviews",
                "place_id": place_id,
                "api_key": api_key,
                "hl": "en",  # Language (optional),
                "sort_by": "qualityScore"
            }

            if next_page_token:
                params["next_page_token"] = next_page_token

            async with session.get("https://serpapi.com/search", params=params) as response:
                results = await response.json()

            if "reviews" in results:
                all_reviews.extend(results["reviews"])

            if "serpapi_pagination" in results and "next_page_token" in results["serpapi_pagination"]:
                next_page_token = results["serpapi_pagination"]["next_page_token"]
            else:
                break

    return all_reviews[:18]


async def apply_review_audit(row, max_reviews):
    reviews = await async_fetch_reviews(row['PlaceID'], serp_api_key, max_reviews)

    # Process reviews directly and return the results
    review_count = 0
    reviews_with_pizza = 0
    for review in reviews:
        try:
            if len(review['snippet']) > 1:
                review_count += 1
                if 'pizza' in review['snippet'].lower() or 'pie' in review['snippet'].lower() or 'pizzeria' in review['snippet'].lower() or 'slice' in review['snippet'].lower():
                    reviews_with_pizza += 1
        except:
            review_count += 0
    return [review_count, reviews_with_pizza]

async def apply_review_audit_async(df,max_reviews):
    tasks = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        task = asyncio.create_task(apply_review_audit(row,max_reviews))
        tasks.append(task)

    results = await asyncio.gather(*tasks)
    return results

**Run Searches Function and Reviews Function separately (to help with debugging / be able to save in case of error later) in batches of 5000 max**

1) Run Searches Function to retrieve API reponse for each shop

In [46]:
# Example usage
df = pd.read_csv('Batch 3.csv')
df['api_response'] = await process_async_search_results(df)


  0%|          | 0/4196 [00:00<?, ?it/s]

2) Run functions on the API response and store in the dataframe

In [49]:
df['PlaceID'] = df['api_response'].apply(get_place_id)
df['Shop Name']=df['api_response'].apply(get_shop_name)
df['Address']=df['api_response'].apply(get_place_address)
df['Email']=df['api_response'].apply(email_lookup)
df['Email1']= df['Email'].apply(lambda x: pd.Series(x))[0]
df['Email2']= df['Email'].apply(lambda x: pd.Series(x))[1]
df['Price']=df['api_response'].apply(price_lookup)
df['Google Classification']=df['api_response'].apply(googleclassification_lookup)
df['Reservation Provider']=df['api_response'].apply(reservation_type)


3) Run the Review Function and store the results in a new column called 'Review_Audit'

In [50]:
df['Review_Audit'] = await apply_review_audit_async(df,18)

  0%|          | 0/4196 [00:00<?, ?it/s]

In [52]:
df

,Account Full ID,Google Search,api_response,PlaceID,Shop Name,Address,Email,Email1,Email2,Price,Google Classification,Reservation Provider,Review_Audit
0,001Ho00001Jfh5zIAB,Mamma Ramona?s Pizzeria 3760 Oceanic Way 92056,{'search_metadata': {'id': '66db4ded88bcde9438...,ChIJjTvO4Tpx3IARzx1r7RXbI5s,Mamma Ramona’s Pizzeria,"3760 Oceanic Way STE 503, Oceanside, CA 92056","([Yelpers report this location has closed], [M...",[Yelpers report this location has closed],"[Mamma Ramona's Pizzeria, Ramona, Ramona]",$10–20,Pizza Takeout,No reservation provider found,"[15, 12]"
1,0011G000013G0vhQAC,Slim and Husky's Pizza 830 Westview Dr SW 30314,{'search_metadata': {'id': '66db4ded9f8bfce1c1...,ChIJN35ULWoD9YgRWPDlBgm_kZA,Slim + Husky's Pizza,"Morehouse College Lower Parking Deck, 830 West...",([834 Westview Drive SW in Atlanta Georgia 303...,[834 Westview Drive SW in Atlanta Georgia 30314],[Get in touch with us today],$10–20,Pizza restaurant,No reservation provider found,"[18, 2]"
2,0011G00000kPXAkQAO,MrJims.Pizza Little Elm 800 W Eldorado Pkwy 7...,{'search_metadata': {'id': '66db4deda505f736db...,ChIJi5lUtMowTIYR7Q5d-qWRI1w,MrJims.Pizza,"800 W Eldorado Pkwy #118, Little Elm, TX 75068","([800 W Eldorado Pkwy , Little Elm, TX 75068],...","[800 W Eldorado Pkwy , Little Elm, TX 75068]","[800 W Eldorado Pkwy, Little Elm, 75068, Email...",$10–20,Pizza delivery,No reservation provider found,"[18, 16]"
3,00137000005uPcxAAE,Mimi's Pizza 2411 N Galloway Ave 75150,{'search_metadata': {'id': '66db4ded48b9f7f7e6...,ChIJceSiMs-lToYRkC5cOamJU2o,Mimi's Pizza,"2411 N Galloway Ave #110, Mesquite, TX 75150","([2411 N Galloway Ave, Mesquite, TX 75150], [2...","[2411 N Galloway Ave, Mesquite, TX 75150]","[2411 N Galloway Ave, Mesquite, TX 75150]",$,Pizza restaurant,No reservation provider found,"[18, 14]"
4,0011G00000ifqYJQAY,Lantana Pizza 467 Greynolds Cir 33462,{'search_metadata': {'id': '66db4ded71b0e847cc...,ChIJlRaHyOLY2IgRBpXh1Jy236A,Lantana Pizza,"467 Greynolds Cir, Lantana, FL 33462","([lantanapizza@live.com], [lantanapizza@live.c...",[lantanapizza@live.com],[lantanapizza@live.com],$10–20,Pizza restaurant,No reservation provider found,"[18, 11]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191,0011G00000tc5cHQAQ,Grub Detroit 17442 Lahser Rd 48219,{'search_metadata': {'id': '66db4e502e33c86e33...,No place found,No place found,No place found,"([17442 Lahser Rd Detroit, MI, 48219-2347], [G...","[17442 Lahser Rd Detroit, MI, 48219-2347]","[Grub Detroit, 17442 Lahser Rd, Detroit, MI 48...",No price found,No Google Classification found,No reservation provider found,"[0, 0]"
4192,0011G00000ifm2TQAQ,Palomar Pizza and Pasta and Salad 3265 Busine...,{'search_metadata': {'id': '66db4e50e1d1ad7ca2...,No place found,No place found,No place found,"([Yelpers report this location has closed], [3...",[Yelpers report this location has closed],"[3265 Business Park Dr, Vista, CA 92081-8552]",No price found,No Google Classification found,No reservation provider found,"[0, 0]"
4193,0011G00000lm98yQAA,Snappers Fish and Chicken and Romeos Pizza 27...,{'search_metadata': {'id': '66db4e50910986c6b1...,ChIJW6oWUx779IgRo0xHEFP4wIg,Snappers Fish & Chicken & Romeos pizza,"27 Lee St Unit A, Jonesboro, GA 30236","([27 Lee St, Jonesboro, GA 30236, romeospizzab...","[27 Lee St, Jonesboro, GA 30236, romeospizzabi...","[27 Lee St, Jonesboro, GA, US]",$10–20,Restaurant,No reservation provider found,"[18, 9]"
4194,0011G00000rQzZyQAK,Gibbys Pizza 37237 S Groesbeck Hwy 48036,{'search_metadata': {'id': '66db4e50a3657a1bf4...,ChIJP5Cn53nfJIgRHJ0mQCmUNnY,Gibbys pizza,"37237 S Groesbeck Hwy, Clinton Twp, MI 48036","([Pizza], [pizza, 37237 S Groesbeck Highway, 4...",[Pizza],"[pizza, 37237 S Groesbeck Highway, 48036]",$,Pizza restaurant,No reservation provider found,"[18, 15]"


4) Run cleanup functions on the output of the main Review Function 

In [53]:
df['Review_counts']= df['Review_Audit'].apply(lambda x: pd.Series(x))[0]
df['Reviews_with_pizza']= df['Review_Audit'].apply(lambda x: pd.Series(x))[1]
df['TP_review_audit']=(df['Reviews_with_pizza']/df['Review_counts']).apply(lambda x: f"{int(np.nan_to_num(x * 100))}%")


df.drop(['api_response','Email','Review_Audit'], axis=1,inplace=True)

5. Save final dataframe to a csv

In [54]:
df.to_csv('Batch 3 - output.csv', index=False)

In [ ]:
from flask import Flask, request, render_template_string
from os import environ

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        nameandaddress = request.form['nameandaddress']
        placeid=get_place_id(nameandaddress)
        reviewsresult=review_audit(placeid,30)
        if placeid is None:
            return render_template_string(f"No place found for given Name and Address")
        if reviewsresult[0]==0:
            return render_template_string(f"No more search credits available or no reviews found")
        return render_template_string(f"Number of reviews scraped is {reviewsresult[0]} and number of reviews with pizza is {reviewsresult[1]}")
    return render_template_string('''
        <form method="POST">
            <input type="text" name="nameandaddress" placeholder="Name and Address">
            <button type="submit">Calculate</button>
        </form>
    ''')

if __name__ == '__main__':
    port = int(environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)